In [ ]:
# Add import statements here
import numpy as np

### The DINAMO-S Algorithm

In [ ]:
class DinamoS():
    def __init__(self, n_bins, eps=0.01):

        self.nb = n_bins
        self.eps = eps
        self.u = None
        self.I = None
        self.sigma_u = None
        self.omega = None
        self.W = None
        self.S_u = None
        self.S_sigma = None
    
    def reference_init(self):
        '''
        Initializes the reference histogram as a uniform distribution (normalized)
        along with initial Poisson uncertainity per bin.

        args:
        n_bins: Number of bins per histogram (default=10)
        '''

        # Uniformly initialize reference histogram.
        u0 = np.random.rand(self.nb)
        # Compute sum of bin-wise frequencies.
        I_u0 = np.sum(u0)
        # Compute bin-wise uncertainty.
        sigma_u0 = np.sqrt((u0 / I_u0) - ((u0 ** 2) / I_u0))

        self.u = u0
        self.I = I_u0
        self.sigma_u = sigma_u0        

    def ewma_init(self, alpha, u, sigma_u):
        self.omega = 1 / sigma_u
        self.W = (1 - alpha) * self.omega
        self.S_u = (1 - alpha) * self.omega * u
        self.S_sigma = (1 - alpha) * self.omega * sigma_u
    
    def chi2_stat(self, x, sigma_x):
        return (1 / self.nb) * np.sum((x - self.u) ** 2 / (sigma_x + self.sigma_u))
    
    def pull_delta(self, x, sigma_x):
        return (x - self.u) / np.sqrt(sigma_x + self.sigma_u)

    def poisson_uncertainty(self, x, I):
        return np.sqrt((x / I) - (x**2 / I))

    def update_reference(self, alpha, xt_i, I, sigma_xi, mu0, W0, S_mu0, S_sigma_mu_0):
        sigma2_xt = self.poisson_uncertainty(xt_i, np.sum(xt_i)) ** 2
        self.omega = 1 / (sigma2_xt + self.eps)
        self.W = alpha * self.W + (1 - alpha) * self.omega
        self.S_u = alpha * self.S_u + (1 - alpha) * self.omega * xt_i
        self.S_sigma = alpha * self.S_sigma + (1 - alpha) * self.omega * (xt_i - self.u) ** 2
        self.u = self.S_u / self.W
        self.sigma_u = np.sqrt(self.S_sigma / self.W)

    def run(self, x, y, alpha, eps):
        mu0, sigma_mu0p, i_mu0 = self.reference_init() #
        W0, S_mu0, S_sigma_mu_0 = self.ewma_init(alpha=alpha, mu0=mu0, sigma_mu0p=sigma_mu0p) #

        chi_hist = list()
        delta_hist = list()
        u_hist = list()
        sigma_u_hist = list()

        for i in range(len(x)):
            x_i = x[i, :]
            y_i = y[i]
            I = np.sum(x_i)
            if I > eps:
                x_it = x_i / I
                sigma_xi = self.poisson_uncertainty(x_it) #
            
            chi2 = None
            pull_delta = None

            if y_i == 0: #
                mu0, sigma_mu0p, W0, S_mu0, S_sigma_mu_0 = self.update_reference(alpha, x_it, I, sigma_xi, mu0, W0, S_mu0, S_sigma_mu_0)
            
            chi_hist.append(chi2)
            delta_hist.append(pull_delta)
            u_hist.append(mu0) #
            sigma_u_hist.append(sigma_mu0p) #